# comparison of accessibility of nucleotides in the sequences

In [30]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'/home/ikea/miniconda2/lib/python2.7/site-packages')


In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [32]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

data for 'tRNA-asp' is corrupted, ignoring...
ADDRSW 	121
GLYCFN 	205
tRNAphe 	76
RNAsep 	154
ZHCV 	95
Z-CIDGMP-1 	135
adenin 	71
23sRNA 	2904
p564 	155
srRNA 	120
ZGLY 	198
MDLOOP 	80
R009 	1542
5srRNA 	170
TRP5 	202


# train predict

In [58]:
import simushape as sim
from rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'])
data.pop("23sRNA")
data.pop("R009")

keys = data.keys()



def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"
predictions = { k: (data[k][1],sim.predict(model,data[k][1])) for k in keys  }
predictions_str = { k: (data[k][1],sim.predict(model,data[k][1], seq_to_db_function=get_str)) for k in keys  }
acc_shaker = {k: rnaplfold(*predictions[k]) for  k in keys} # a predcition is seq,rea
acc_shaker_str = {k: rnaplfold(*predictions_str[k]) for  k in keys} # a predcition is seq,rea


data for 'tRNA-asp' is corrupted, ignoring...


In [59]:
if False:
    data = getdata()
    keys = data.keys()
    shaker_shape = sim.crosspredict_nfold(data,keys)
    

# real and sukosd based accessibility

In [60]:
from rna_tools.sukosd import sukosd
acc_suko =   {k: rnaplfold(data[k][1], sukosd(data[k][2]) ) for  k in keys}
acc_shape =  {k: rnaplfold(data[k][1],data[k][0]) for  k in keys}  
acc_nodata = {k: rnaplfold(data[k][1]) for  k in keys}


In [61]:
from scipy.stats import spearmanr as spear
corr = lambda ac1, ac2:[spear(ac1[k], ac2[k])[0] for k in keys]
corr_suko =  corr(acc_suko, acc_shape)
corr_shaker =  corr(acc_shaker, acc_shape)
corr_shaker_str =  corr(acc_shaker_str, acc_shape)
corr_nodata =  corr(acc_nodata, acc_shape)

# Visualize 

In [62]:
import pandas
import numpy as np
index= ['suko',"shaker","shaker_str","nodata"]
data2 = [corr_suko,corr_shaker, corr_shaker_str,corr_nodata]
map(lambda x:x.append(np.mean(x)),data2)

data2 = pandas.DataFrame(data2, columns=keys+['mean'], index=index).T

from tabulate import tabulate
print tabulate(data2, headers="keys", tablefmt="orgtbl")

|            |     suko |   shaker |   shaker_str |   nodata |
|------------+----------+----------+--------------+----------|
| ADDRSW     | 0.973697 | 0.983577 |     0.983979 | 0.911415 |
| GLYCFN     | 0.295878 | 0.528483 |     0.258756 | 0.520805 |
| tRNAphe    | 0.91992  | 0.905508 |     0.945657 | 0.631888 |
| RNAsep     | 0.937153 | 0.958294 |     0.962743 | 0.847748 |
| ZHCV       | 0.843379 | 0.855105 |     0.877968 | 0.905179 |
| Z-CIDGMP-1 | 0.936089 | 0.972595 |     0.964446 | 0.978766 |
| adenin     | 0.861568 | 0.963031 |     0.962662 | 0.877654 |
| p564       | 0.961632 | 0.796816 |     0.951913 | 0.793642 |
| srRNA      | 0.92327  | 0.490149 |     0.977448 | 0.55039  |
| ZGLY       | 0.918547 | 0.740316 |     0.931708 | 0.860329 |
| MDLOOP     | 0.968401 | 0.978238 |     0.977236 | 0.948119 |
| 5srRNA     | 0.788022 | 0.980049 |     0.764599 | 0.978867 |
| TRP5       | 0.907063 | 0.973273 |     0.974894 | 0.975396 |
| mean       | 0.864201 | 0.855803 |     0.887231 | 0.8

In [64]:
print data2.to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &      suko &    shaker &  shaker\_str &    nodata \\
\midrule
ADDRSW     &  0.973697 &  0.983577 &    0.983979 &  0.911415 \\
GLYCFN     &  0.295878 &  0.528483 &    0.258756 &  0.520805 \\
tRNAphe    &  0.919920 &  0.905508 &    0.945657 &  0.631888 \\
RNAsep     &  0.937153 &  0.958294 &    0.962743 &  0.847748 \\
ZHCV       &  0.843379 &  0.855105 &    0.877968 &  0.905179 \\
Z-CIDGMP-1 &  0.936089 &  0.972595 &    0.964446 &  0.978766 \\
adenin     &  0.861568 &  0.963031 &    0.962662 &  0.877654 \\
p564       &  0.961632 &  0.796816 &    0.951913 &  0.793642 \\
srRNA      &  0.923270 &  0.490149 &    0.977448 &  0.550390 \\
ZGLY       &  0.918547 &  0.740316 &    0.931708 &  0.860329 \\
MDLOOP     &  0.968401 &  0.978238 &    0.977236 &  0.948119 \\
5srRNA     &  0.788022 &  0.980049 &    0.764599 &  0.978867 \\
TRP5       &  0.907063 &  0.973273 &    0.974894 &  0.975396 \\
mean       &  0.864201 &  0.855803 &    0.887231 &  0.829246 \\
\botto